In [1]:
"""
Put an .env in the root of your Google Drive. The path to the .env file should be /content/drive/MyDrive/envFile/.env
When you mount your Drive, you can access the .env and import it into this notebook. You can put your service key .json
file in the same directory if you are using a service account.

Example of an .env file:

PROJECT_ID=<your project id>
REGION=us-central1
BUCKET=<your staging bucket name>
DOCUMENT_PATH=<path to your pdfs>
DEPLOYED_API_ENDPOINT_ID=<This is all numbers>
INDEX_ENDPOINT=<not used>
DEPLOYED_INDEX_ID=<This is all numbers>
SVC_APIKEY_PATH=<path to your service account key>
SVC_ACCOUNT=example-service-account@<your project id>.iam.gserviceaccount.com
ENDPOINT_NAME=<your endpoint name>
VECTOR_BUCKET=<your vector bucket name>
"""

'\nPut an .env in the root of your Google Drive. The path to the .env file should be /content/drive/MyDrive/envFile/.env\nWhen you mount your Drive, you can access the .env and import it into this notebook. You can put your service key .json\nfile in the same directory if you are using a service account.\n\nExample of an .env file:\n\nPROJECT_ID=<your project id>\nREGION=us-central1\nBUCKET=<your staging bucket name>\nDOCUMENT_PATH=<path to your pdfs>\nDEPLOYED_API_ENDPOINT_ID=<This is all numbers>\nINDEX_ENDPOINT=<not used>\nDEPLOYED_INDEX_ID=<This is all numbers>\nSVC_APIKEY_PATH=<path to your service account key>\nSVC_ACCOUNT=example-service-account@<your project id>.iam.gserviceaccount.com\nENDPOINT_NAME=<your endpoint name>\nVECTOR_BUCKET=<your vector bucket name>\n'

Set up development system

In [2]:
# Moving all runtime dependencies to another notebook. This notebook is for endpoint creation only.
import os
import sys
import IPython
from io import BytesIO

packages = [
    "langchain",
    "langchain_community",
    "langchain-google-vertexai",
    "google-cloud-aiplatform",
    "google-auth==2.38.0",
    "vertexai",
    "python-dotenv",
    "pypdf2"
]

packages_str = " ".join(packages)
print(f"Installing all packages: {packages_str}")

IPython.get_ipython().run_line_magic("pip", f"install -qU {packages_str}")

Installing all packages: langchain langchain_community langchain-google-vertexai google-cloud-aiplatform google-auth==2.38.0 vertexai python-dotenv pypdf2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [3]:
# Install torch separately
pytorch_command = "torch --index-url https://download.pytorch.org/whl/cu118"
print(f"Installing PyTorch with CUDA: {pytorch_command}")
IPython.get_ipython().run_line_magic("pip", f"install -qU --no-input {pytorch_command}")

Installing PyTorch with CUDA: torch --index-url https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [4]:
# Mount our Google Drive and authenticate
# In the root directory of your drive, create a folder called "envFile"
# Inside of the folder, create a .env file and create for any sensitive information like project ids.
# The full file path should be /content/drive/MyDrive/envFile/.env
# Every time your team runs the notebook, it will mount your Google Drive and everyone has the same
# directory structues with the same .env file.
try:
    from google.colab import drive, auth
    drive.mount('/content/drive', force_remount=True)
    # drive.mount('/content/drive/MyDrive/envFile/.env', force_remount=True)
    auth.authenticate_user()
except ImportError:
    pass

Mounted at /content/drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from google.oauth2 import service_account
from google.cloud import aiplatform, storage
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    Namespace,
    NumericNamespace,
)

In [7]:
import langchain
from langchain_google_vertexai import (
    VectorSearchVectorStore,
    VectorSearchVectorStoreDatastore,
    VertexAIEmbeddings,
    VertexAI
)
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [8]:
# Create your own API key or comment this out.
# from google.colab import userdata
# userdata.get('VertexAPI')
# !export GOOGLE_APPLICATION_CREDENTIALS="/content/drive/MyDrive/envFile/svc-fishbowlAPI-5560.json"
# Imports
import textwrap
import json
import PyPDF2
import vertexai
from dotenv import load_dotenv

In [9]:
# Restart the kernel <---- Not needed anymore, but keeping it for a rainy day
# if "google.colab" in sys.modules:
#     ip_app = IPython.Application.instance()
#     ip_app.kernel.do_shutdown(True)

In [10]:
# Load environmental variables

dotenv_path = '/content/drive/MyDrive/envFile/.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [11]:
# store the sensitive information into variables.
project_id = os.environ.get("PROJECT_ID")
region = os.environ.get("REGION")
bucket = os.environ.get("BUCKET")
data_path = os.environ.get("DOCUMENT_PATH")
api_endpoint_id = os.environ.get("DEPLOYED_API_ENDPOINT_ID")
index_endpoint = os.environ.get("INDEX_ENDPOINT")
deployed_index_id = os.environ.get("DEPLOYED_INDEX_ID")
svc_apikey_path = os.environ.get("SVC_APIKEY_PATH")
svc_account = os.environ.get("SVC_ACCOUNT")
vector_db = os.environ.get("VECTOR_BUCKET")
staging_bucket_uri = f"gs://{bucket}/{data_path}"
vector_bucket_uri = f"gs://{vector_db}/{data_path}"

In [12]:
# debugging
!echo $PROJECT_ID, $REGION, $BUCKET, $DEPLOYED_API_ENDPOINT_ID, $DEPLOYED_INDEX_ID

myadta5560project, us-central1, global-bucket-5560, 5151550625711915008, 3573678872085921792


In [13]:
# Load the secret
if not svc_apikey_path:
    print("Error: APIKEY environment variable not found in .env file.")
else:
    print(f"Service Account key path loaded: {svc_apikey_path}")

    try:
        credentials = service_account.Credentials.from_service_account_file(svc_apikey_path)
        aiplatform.init(project=project_id, location=region, staging_bucket=staging_bucket_uri)
        print("Vertex AI client initialized with credentials from file path.")
    except Exception as e:
        print(f"Error initializing Vertex AI: {e}")

Service Account key path loaded: /content/drive/MyDrive/envFile/svc-fishbowlAPI-5560.json
Vertex AI client initialized with credentials from file path.


In [14]:
embedding_model = VertexAIEmbeddings(model_name="text-embedding-005")
DISPLAY_NAME = "langchain-index"
DEPLOYED_INDEX_ID = "langchain_index_id"
DIMENSIONS = 768

existing_indices = aiplatform.MatchingEngineIndex.list(filter=f'display_name="{DISPLAY_NAME}"')

# Prevent recreation or accidental creation of resources
if existing_indices:
    ai_idx = existing_indices[0]
    print(f"MatchingEngineIndex '{DISPLAY_NAME}' already exists with ID: {ai_idx.name}")
else:
    ai_idx = aiplatform.MatchingEngineIndex.create_tree_ah_index(
        display_name=DISPLAY_NAME,
        dimensions=DIMENSIONS,
        approximate_neighbors_count=150,
        distance_measure_type="DOT_PRODUCT_DISTANCE",
        index_update_method="STREAM_UPDATE",  # allowed values BATCH_UPDATE , STREAM_UPDATE
    )
    print(f"Created MatchingEngineIndex '{DISPLAY_NAME}' with ID: {ai_idx.name}")

ENDPOINT_DISPLAY_NAME = f"{DISPLAY_NAME}-endpoint"
existing_endpoints = aiplatform.MatchingEngineIndexEndpoint.list(filter=f'display_name="{ENDPOINT_DISPLAY_NAME}"')

if existing_endpoints:
    ai_index_endpoint = existing_endpoints[0]
    print(f"MatchingEngineIndexEndpoint '{ENDPOINT_DISPLAY_NAME}' already exists with ID: {ai_index_endpoint.name}")
else:
    ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
        display_name=ENDPOINT_DISPLAY_NAME, public_endpoint_enabled=True
    )
    print(f"Created MatchingEngineIndexEndpoint '{ENDPOINT_DISPLAY_NAME}' with ID: {ai_index_endpoint.name}")

deployed_index_found = False
if ai_index_endpoint.deployed_indexes:
    for deployed_index in ai_index_endpoint.deployed_indexes:
        if deployed_index.id == DEPLOYED_INDEX_ID and deployed_index.index == ai_idx.resource_name:
            print(f"Index '{ai_idx.display_name}' with ID '{ai_idx.name}' is already deployed to endpoint '{ai_index_endpoint.display_name}' with deployed ID '{DEPLOYED_INDEX_ID}'.")
            deployed_index_found = True
            break

if not deployed_index_found:
    # NOTE : This operation can take upto 20 minutes
    ai_index_endpoint = ai_index_endpoint.deploy_index(
        index=ai_idx, deployed_index_id=DEPLOYED_INDEX_ID
    )
    print(f"Deployed index '{ai_idx.display_name}' with ID '{ai_idx.name}' to endpoint '{ai_index_endpoint.display_name}' with deployed ID '{DEPLOYED_INDEX_ID}'.")

MatchingEngineIndex 'langchain-index' already exists with ID: 3573678872085921792
MatchingEngineIndexEndpoint 'langchain-index-endpoint' already exists with ID: 5151550625711915008
Index 'langchain-index' with ID '3573678872085921792' is already deployed to endpoint 'langchain-index-endpoint' with deployed ID 'langchain_index_id'.


In [15]:
print("\n--- Existing Matching Engine Resources ---")

print("\nMatching Engine Indices:")
list_idx = aiplatform.MatchingEngineIndex.list()
for idx in list_idx:
    print(f"  Display Name: {idx.display_name}, ID: {idx.name}")

print("\nMatching Engine Index Endpoints:")
list_endpoints = aiplatform.MatchingEngineIndexEndpoint.list()
for endpoint in list_endpoints:
    print(f"  Display Name: {endpoint.display_name}, ID: {endpoint.name}, Deployed Indexes: {endpoint.deployed_indexes}")


--- Existing Matching Engine Resources ---

Matching Engine Indices:
  Display Name: langchain-index, ID: 3573678872085921792

Matching Engine Index Endpoints:
  Display Name: langchain-index-endpoint, ID: 5151550625711915008, Deployed Indexes: [id: "langchain_index_id"
index: "projects/872233283772/locations/us-central1/indexes/3573678872085921792"
create_time {
  seconds: 1744688122
  nanos: 79276000
}
index_sync_time {
  seconds: 1745280976
  nanos: 7075000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]


In [16]:
# List existing Matching Engine Indices
indices = aiplatform.MatchingEngineIndex.list(
    project=project_id, location=region
)
for index in indices:
    print(f"Index Display Name: {index.display_name}")
    print(f"Index ID: {index.name}")
    print(f"Index Resource Name: {index.resource_name}")
    print("-" * 20)

# List existing Matching Engine Index Endpoints
endpoints = aiplatform.MatchingEngineIndexEndpoint.list(
    project=project_id, location=region
)
for endpoint in endpoints:
    print(f"Endpoint Display Name: {endpoint.display_name}")
    print(f"Endpoint ID: {endpoint.name}")
    print(f"Endpoint Resource Name: {endpoint.resource_name}")
    print("Deployed Indexes:")
    for deployed_index in endpoint.deployed_indexes:
        print(f"  - Deployed Index ID: {deployed_index.id}")
        print(f"  - Index Resource Name: {deployed_index.index}")
    print("-" * 20)

Index Display Name: langchain-index
Index ID: 3573678872085921792
Index Resource Name: projects/872233283772/locations/us-central1/indexes/3573678872085921792
--------------------
Endpoint Display Name: langchain-index-endpoint
Endpoint ID: 5151550625711915008
Endpoint Resource Name: projects/872233283772/locations/us-central1/indexEndpoints/5151550625711915008
Deployed Indexes:
  - Deployed Index ID: langchain_index_id
  - Index Resource Name: projects/872233283772/locations/us-central1/indexes/3573678872085921792
--------------------


In [17]:
# Load pdfs
def load_text_from_pdf(bucket_name: str, prefix: str = ""):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    documents = []

    for blob in blobs:
        if blob.name.lower().endswith(".pdf"):
            try:
                blob_data = blob.download_as_bytes()
                pdf_file = BytesIO(blob_data)
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text_content = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text_content += page.extract_text()

                metadata = {"file_name": blob.name}
                documents.append({"content": text_content, "metadata": metadata})

            except PyPDF2.errors.PdfReadError as e:
                print(f"Error reading PDF file {blob.name}: {e}")
            except Exception as e:
                print(f"Error processing file {blob.name}: {e}")
        else:
            print(f"Skipping non-PDF file: {blob.name}")

    return documents

In [18]:
if __name__ == "__main__":
    bucket_name = bucket
    data = data_path

    loaded_files = load_text_from_pdf(bucket_name, prefix=data)
    # debugging
    # for doc in loaded_files:
    #     print(f"File: {doc['metadata']['file_name']}")
    #     print(f"Content preview: {doc['content'][:10]}...\n")

Skipping non-PDF file: documents/data.zip
Skipping non-PDF file: documents/data/


In [19]:
try:
  vector_store = VectorSearchVectorStore.from_components(
      project_id=project_id,
      region=region,
      gcs_bucket_name=vector_db,
      gcs_bucket_uri=f"gs://{vector_db}",
      index_id=deployed_index_id,
      endpoint_id=api_endpoint_id,
      embedding=embedding_model,
      stream_update=True,
  )
  print("VectorSearchVectorStore initialized.")

  # Prepare documents for the vector store
  documents_to_store = [
      Document(page_content=doc['content'], metadata=doc['metadata'])
      for doc in loaded_files
  ]

  # Add documents to the vector store
  print("Adding documents to the vector store...")
  vector_store.add_documents(documents=documents_to_store)
  print("Documents added to the vector store.")

except Exception as e:
    print(f"Error creating or adding to VectorSearchVectorStore: {e}")

VectorSearchVectorStore initialized.
Adding documents to the vector store...
Documents added to the vector store.


In [20]:
vector_store.similarity_search("What is picking?, k=2")

[Document(metadata={'file_name': 'documents/data/scraped_data_https___help.fishbowlinventory.com_advanced_s_article_Picking.pdf'}, page_content="url heading content images\nhttps://\nhelp.fishbowlinventory.com/\nadvanced/s/article/PickingFishbowl\nAdvanced -\nPicking[]\nhttps://\nhelp.fishbowlinventory.com/\nadvanced/s/article/PickingDec 17,\n2024Knowledge[]\nhttps://\nhelp.fishbowlinventory.com/\nadvanced/s/article/PickingArticle\nDetailsThe Picking\nmodule, located\nin the Sales\ngroup ,\nfacilitates the\nprocess of\npicking (or\ngathering) items\nfrom their\nstorage\nlocations.\nTypically, after\nan item has been\npicked, it will\nbe ready to be\npacked and\nshipped in the\nShipping module .\nFishbowl will\nalso create picks\nwhen inventory\nneeds to be\ngathered for a\nwork order, a\ntransfer order,\nor a credit\nreturn purchase\norder.[]\nhttps://\nhelp.fishbowlinventory.com/\nadvanced/s/article/PickingContents []\nhttps://\nhelp.fishbowlinventory.com/\nadvanced/s/article/PickingM

In [21]:
class LLMHandler:
    def __init__(self, model_name="gemini-2.5-pro-exp-03-25", max_output_tokens=8192, temperature=0.2, top_p=0.8, top_k=40, verbose=True):
        self.llm = VertexAI(
            model_name=model_name,
            max_output_tokens=max_output_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            verbose=verbose,
        )

In [22]:
class RetrieverHandler:
    def __init__(self, vector_db, num_results=10, search_distance_threshold=0.6):
        self.vector_db = vector_db
        self.num_results = num_results
        self.search_distance_threshold = search_distance_threshold
        self.retriever = self.vector_db.as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": self.num_results,
                "search_distance": self.search_distance_threshold,
            },
            filters=None,
        )

In [23]:
class QAChainHandler:
    def __init__(self, llm_handler, retriever_handler, template, verbose=False):
        self.qa = RetrievalQA.from_chain_type(
            llm=llm_handler.llm,
            chain_type="stuff",
            retriever=retriever_handler.retriever,
            return_source_documents=False,
            verbose=verbose,
            chain_type_kwargs={
                "prompt": PromptTemplate(
                    template=template,
                    input_variables=["context", "question"],
                ),
            },
        )
        if verbose:
            self.qa.combine_documents_chain.verbose = True
            self.qa.combine_documents_chain.llm_chain.verbose = True
            self.qa.combine_documents_chain.llm_chain.llm.verbose = True

In [24]:
class OutputFormatter:
    @staticmethod
    def format_result(result):
        print(f"Query: {result['query']}")
        print("-" * 80)
        if "source_documents" in result.keys():
            for idx, ref in enumerate(result["source_documents"]):
                print("*" * 80)
                print(f"REFERENCE #{idx}")
                print("*" * 80)
                if "score" in ref.metadata:
                    print(f"Matching Score: {ref.metadata['score']}")
                if "source" in ref.metadata:
                    print(f"Document Source: {ref.metadata['source']}")
                if "document_name" in ref.metadata:
                    print(f"Document Name: {ref.metadata['document_name']}")
                print("*" * 80)
                print(f"Content: \n{OutputFormatter.wrap(ref.page_content)}")
                print("*" * 80)
        print(f"Response: {OutputFormatter.wrap(result['result'])}")
        print("-" * 80)

    @staticmethod
    def wrap(s):
        return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))

In [25]:
def ask(query, qa_handler, num_results=10, search_distance=0.6, filters={}):
    qa_handler.qa.retriever.search_kwargs["search_distance"] = search_distance
    qa_handler.qa.retriever.search_kwargs["k"] = num_results
    qa_handler.qa.retriever.search_kwargs["filters"] = filters
    result = qa_handler.qa({"query": query})
    OutputFormatter.format_result(result)

In [26]:
template = """
SYSTEM: You are an intelligent assistant helping the users with a new inventory management software called Fishbowl

Question: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
- If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
- If the context is empty, just say "I do not know the answer to that."
=========
{context}
=========
Question: {question}
Helpful Answer:"""

In [27]:
# The responses are long because verbosity is set to True by default. When you initialize the llm_handler, you can
# set it to false. On the QAChainHandler class, you'll have to set the return_source_documents=False

# LLMHandler(
    # model_name="gemini-2.5-pro-exp-03-25",
    # max_output_tokens=8192,
    # temperature=0.2,
    # top_p=0.8,
    # top_k=40,
    # verbose=True <------
    # )



llm_handler = LLMHandler(verbose=False) # <----- Change it here
retriever_handler = RetrieverHandler(vector_store)
qa_handler = QAChainHandler(llm_handler, retriever_handler, template, verbose=False) # <----- Change it here also

In [28]:
# Ask a question, This one is intended to fail. You should get a response "I cannot determine the answer to that."

ask("What is the meaning of life?", qa_handler)

<ipython-input-25-ff939536cdf2>:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_handler.qa({"query": query})


Query: What is the meaning of life?
--------------------------------------------------------------------------------
Response: I cannot determine the answer to that.
--------------------------------------------------------------------------------


In [29]:
ask("What are the different modules included with Fishbowl?", qa_handler)

Query: What are the different modules included with Fishbowl?
--------------------------------------------------------------------------------
Response: Based on the provided context:  Fishbowl is organized into modules, which are accessed by selecting a Module Group and
then clicking a Module.  Examples mentioned include: *   The **My Fishbowl** module (located in the General group). *
The **Sales Order** module (located in the Sales group). *   The **Accounting** module.  The context also mentions an
image that gives an overview of the major modules, but the image itself is not included in the text provided. Therefore,
a complete list of all modules included with Fishbowl cannot be determined from the context alone.
--------------------------------------------------------------------------------


In [30]:
ask("Where can I get Customer Support information?", qa_handler)

Query: Where can I get Customer Support information?
--------------------------------------------------------------------------------
Response: Based on the provided context:  The **My Fishbowl** module, located in the **General** group, displays helpful
information related to support:  1.  It shows **support contract details**. 2.  The **Support tab** within the My
Fishbowl module displays the support tickets created when your company contacts Fishbowl. You can also create a new
request by clicking the **New Support Request** button on this tab. 3.  The **Company tab** displays the contact
information that Fishbowl will use to contact your company. 4.  The **Contract tab** can be used to renew your support
contract.
--------------------------------------------------------------------------------


In [31]:
ask("Where can I get the user manual?", qa_handler)

Query: Where can I get the user manual?
--------------------------------------------------------------------------------
Response: Based on the provided context, the Fishbowl Advanced documentation can be found at:
https://help.fishbowlinventory.com/advanced/s/article/Documentation
--------------------------------------------------------------------------------


In [36]:
ask("Can you give me a list of items with low inventory levels?", qa_handler)

Query: Can you give me a list of items with low inventory levels?
--------------------------------------------------------------------------------
Response: Based on the provided context:  The Inventory Dashboard module includes graphs for "short parts," which may relate to
low inventory levels. The Inventory module allows you to view parts and their quantities on hand.  However, the context
does not explicitly describe how to generate a specific *list* of items identified as having low inventory levels.
Therefore, I cannot determine the answer to that.
--------------------------------------------------------------------------------


In [37]:
ask("Can you provide the link for Inventory Dashboard?", qa_handler)

Query: Can you provide the link for Inventory Dashboard?
--------------------------------------------------------------------------------
Response: Here is the link for the Inventory Dashboard: https://help.fishbowlinventory.com/advanced/s/article/Inventory-Dashboard
--------------------------------------------------------------------------------


In [38]:
ask("Can you provide information on pricing module?", qa_handler)

Query: Can you provide information on pricing module?
--------------------------------------------------------------------------------
Response: The Pricing Rule module, found in the Sales group, enables automatic price adjustments on sales orders based on various
factors like average cost, order quantity, customer details, day, or specific products.  Here's what you can do with the
Pricing Rule module:  1.  **Create Rules:** Use the 'New' button to create a pricing rule. You select applicable
customers and products, then choose an adjustment type and a price/cost value. 2.  **Manage Rules:** You can Save,
Delete, or Duplicate existing rules. Rules can also be inactivated using the 'Active' checkbox on the General tab. 3.
**Adjust Product Prices:** Besides affecting sales order prices, rules can directly adjust the 'Price' field within the
Product module. If a rule is only for adjusting product prices, it can be disabled for sales orders via the Advanced
tab. 4.  **Understand Precedenc

In [39]:
ask("Where can i get details about my purchase order?", qa_handler)

Query: Where can i get details about my purchase order?
--------------------------------------------------------------------------------
Response: You can get details about your purchase order within the **Purchase Order module**.  Once you have located your purchase
order (using the Search pane on the left if needed), you can find information in the following tabs:  1.  **General
tab:** Displays important order information like the vendor, order number, order status, addresses, order items, and
more. 2.  **Details tab:** Shows important order details such as a link to the vendor, the last user who modified the
order, shipping terms, payment terms, carrier, order dates, and more. 3.  **Memo tab:** Allows you to view any memos
added to the purchase order, including the date they were created and the user who created them. 4.  **Custom tab:**
Stores additional custom fields specific to the order.
--------------------------------------------------------------------------------


In [40]:
ask("Provide details about Fishbowl Flow Chart", qa_handler)

Query: Provide details about Fishbowl Flow Chart
--------------------------------------------------------------------------------
Response: I cannot determine the answer to that.
--------------------------------------------------------------------------------


In [41]:
ask("Where can I do Capacity Planning?", qa_handler)

Query: Where can I do Capacity Planning?
--------------------------------------------------------------------------------
Response: Based on the provided context:  Capacity Planning can be done using the **Capacity Planning Report** and the **Category
Capacity Planning** report. These reports are located within the **BI Report module**, which is part of the **Reporting
group**.  *   The **Capacity Planning Report** displays scheduled work orders, total time scheduled per day, and allows
rescheduling of work orders. *   The **Category Capacity Planning** report displays weekly capacity and scheduled work
orders separated by calendar category.
--------------------------------------------------------------------------------


In [42]:
ask("How does Fishbowl yse import and export documentation?", qa_handler)

Query: How does Fishbowl yse import and export documentation?
--------------------------------------------------------------------------------
Response: The list of imports and exports, along with the corresponding documentation, can be found in the Fishbowl Client. This
documentation includes instructions for the available imports and exports.
--------------------------------------------------------------------------------


In [43]:
ask("I want to know about the YTD Revenues", qa_handler)

Query: I want to know about the YTD Revenues
--------------------------------------------------------------------------------
Response: I cannot determine the answer to that.  While the context mentions several reports related to revenue and sales, such as
the MTD Chart (Month to Date), Sales By Region, Sales By Rep, Sales Goal Chart, Top Products Chart, Gross Sales reports,
and TPS Report, none are explicitly described as providing "YTD Revenues" (Year to Date Revenues). Some reports can be
filtered by date range, which might allow for calculating YTD figures, but the context does not explicitly confirm a
dedicated YTD Revenue report.
--------------------------------------------------------------------------------


In [44]:
ask("I want to create a new kit, where can I get information on the same? ", qa_handler)

Query: I want to create a new kit, where can I get information on the same? 
--------------------------------------------------------------------------------
Response: Based on the provided context:  1.  **Locate the Module:** Kits are created in the **Product module**. 2.  **Initiate
Creation:** To create a new kit, go to the Product module, click the small arrow on the **New** button, and select **New
Kit**. 3.  **Enter General Information:** A blank kit will appear. Enter the Kit Name, Description, Price, etc., in the
**General** tab. 4.  **Add Kit Items:** Select the **Kit Items** tab. Click the **Add Product to Kit** icon on the
component toolbar. This opens the **Add Kit Item Wizard**. 5.  **Choose Item Type:** The wizard will first ask you to
choose the type of item to add (Standard, Optional, Variable, Variable-Optional, Discount, or Tax Rate). 6.  **Follow
Wizard Steps:** The process for adding items is similar for each type, though the wizard steps might vary slightly.
Ensure

Notes and Links to resources

In [32]:
# TODO: Get api keys https://docs.smith.langchain.com/administration/how_to_guides/organization_management/create_account_api_key
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

In [33]:
# https://docs.smith.langchain.com/prompt_engineering/quickstarts/quickstart_sdk

In [34]:
# https://python.langchain.com/docs/integrations/chat/

In [35]:
# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/